<img src='./figures/logo-ecole-polytechnique-ve.jpg' style='position:absolute; top:0; right:0;' width='100px' height='' alt='' />

<center>**Ecole Polytechnique, Cycle ingénieur**</center>
<center>MAP572</center>

# Résolution de k-SAT par Monte-Carlo

In [90]:
# css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./style/custom2.css").read()
    return HTML(styles)
#css_styling()

In [91]:
# load the libraries
import matplotlib.pyplot as plt # 2D plotting library
import numpy as np              # package for scientific computing  
import random
%matplotlib inline  

## Table des matières

- [k-SAT et exploration aléatoire](#kSAT)
  - k-SAT : Définition et Préliminaires
  - [Résolution de $2$-SAT avec <i>WalkSat</i>](#WalkSat)
  - [<i>WalkSat</i> : choix du paramètre $T$](#ChoixT)  
- [Matrices de transitions : Calcul de $\mathbb{P}(\tau < 2n^2)$](#Annexe)
- [Application : Transition de phase pour $2$-SAT](#Transition)


Pour un problème sur un espace fini, les méthodes <i>Monte-Carlo Markov Chain</i> (MCMC) consistent à parcourir l'ensemble des solutions possibles de façon aléatoire mais astucieuse. On cherche ainsi à déterminer la solution optimale, ou proche de l'optimal.

L'objectif de ce TP est d'illustrer la puissance de la stratégie MCMC sur un problème particulier : le problème $k$-SAT en informatique théorique. C'est aussi un prétexte pour utiliser les matrices de transitions.

Une référence pour ce TP est :<br>
[1] MITZENMACHER, Michael et UPFAL, Eli. Probability and computing: Randomization and probabilistic techniques in algorithms and data analysis. Cambridge university press, 2017. (pages 156-159)


<a id="kSAT"></a>
# Le problème k-SAT

Le problème SAT en informatique théorique (aussi appelé problème de satisfaisabilité booléenne) est le problème de décision qui, étant donné une formule de logique booléenne, détermine s'il existe une assignation des variables qui rend la formule vraie.

Nous allons définir tous les termes et restreindre le contexte.

Ici on va spécifiquement s'intéresser au problème $k$-SAT. Soit $k$ fixé (dans ce TP on va prendre uniquement $k\in \{2,3\})$ et étant données $n$ variables booléennes $x_1,x_2,\dots,x_n \in \{\text{Vrai},\text{Faux}\}$, on considère les formules booléennes de la forme ($\vee$ signifie 'ou' et $\wedge$ signifie 'et')<br><br>

$$
(z_{1,1} \vee z_{1,2} \vee \dots \vee z_{1,k}) \wedge \dots \wedge (z_{M,1} \vee z_{M,2} \vee \dots \vee z_{M,k})
$$

<br>
où

* $M$ est un entier quelconque 
* pour chaque $1\leq m\leq M$ et chaque $i\leq k$ on a

<br>

$$
z_{m,i}\in \big\{x_1,x_2,\dots,x_n,\overline{x_1},\overline{x_2},\dots,\overline{x_n}\big\}.
$$

(La notation $\overline{x}$ désigne la négation de $x$.)
Chaque terme $(z_{m,1} \vee z_{m,2} \vee \dots \vee z_{m,k})$ est appelée une clause. 

Par exemple pour $2$-SAT avec $n=5$ variables, une formule à $M=3$ clauses est donnée par
<br><br>

$$
F=(x_1 \vee \overline{x_5}) \wedge (x_2 \vee x_3)  \wedge (\overline{x_1} \vee x_2) 
$$

<br>

Une <i>affectation</i> est une fonction $x_1,x_2,\dots,x_n \in \{\text{Vrai},\text{Faux}\}$. S'il existe une affectation qui rende $F$ vraie on dit que $F$ est **satisfiable**.

Dans le cas de l'exemple ci-dessus, une <i>affectation</i> des variables qui rende $F$ vraie est :

$$
x_1=\text{Faux},\ x_2=\text{Vrai},\ x_3=\text{Faux},\ x_4=\text{Faux},\ x_5=\text{Faux}.
$$

Spécifiquement, le problème $k$-SAT est de trouver un algorithme qui, étant donnée une formule $F$, trouve une affectation des variables qui rende $F$ vraie (ou qui retourne `impossible` si une telle affectation n'existe pas). Le problème 2-SAT est polynomial alors que 3-SAT est NP-complet (voir par exemple : S.Perifel. Complexité algorithmique. Ellipses (2014))

<div markdown=1 class="DoIt"> 

Ecrire des fonctions `ClauseVraie(Clause,Affectation,nb_variables)` et `FonctionVraie(Formule,Affectation,nb_variables)` qui prennent en entrée des clauses ou formules et une affectation, et calcule si la clause/formule est vraie ou pas.

On pourra représenter les formules sous la forme de liste :
```
Formule = [Clause1,Clause2,...,ClauseM]
```
où chaque `Clause` est de la forme
```
Clause = [z_1,...,z_k]
```
et chaque $z_i$ est un booléen. Dans les clauses on peut numéroter les booléens de $0$ à $n-1$ pour $x_1, \dots,x_n$ et de $n$ à $2n-1$ pour leurs négations. Pour l'exemple plus haut :
```
F=[[0,9],[1,2],[5,1]]
```
car (par exemple) $\overline {x_5}$ est codé par $9$. L'affectation donnée en exemple est alors
```
[0,1,0,0,0]
```

In [92]:
l=[0,1]
isinstance(l,list)

True

In [93]:
####################################
# ---- tester une clause
####################################

def ClauseVraie(Clause,Affectation,nb_variables):
    for i in range(len(Clause)):
        v = Clause[i]
        if v < nb_variables:
            if Affectation[v]==1:
                return True
        if v>= nb_variables:
            if Affectation[v-nb_variables]==0:
                return True
    return False


# Test
n=2
Clause1=[0,1]  # x_1 ou x_2
Clause2=[2,1]  # non(x_1) ou x_2
Clause3=[2,3]  # non(x_1) ou non(x_2)
        
Affectation1=[1,0] # x_1 vraie et x_2 fausse

print(ClauseVraie(Clause1,Affectation1,n)) # doit renvoyer True
print(ClauseVraie(Clause2,Affectation1,n)) # doit renvoyer False
print(ClauseVraie(Clause3,Affectation1,n)) # doit renvoyer True

True
False
True


In [94]:
####################################
# ---- tester une formule
####################################

def FormuleVraie(Formule,Affectation,nb_variables):
    clauseVars = []
    for i in range(len(Formule)):
        c = Formule[i]
        if not ClauseVraie(c,Affectation,nb_variables):
            return False
    return True

# test
Formule1=[Clause1,Clause2]
Formule2=[Clause1,Clause3]
Formule3=[Clause2,Clause3]
print([FormuleVraie(f,Affectation1,2) for f in [Formule1,Formule2,Formule3]]) # Doit renvoyer [False, True, False]

# test
Formule=[[0,9],[1,2],[5,1]]
Affectation=[0,1,0,0,0]
print(FormuleVraie(Formule,Affectation,5)) # Doit renvoyer True

[False, True, False]
True


<a id="WalkSat"></a>
## 2-SAT : l'algorithme WalkSat

Le problème 2-SAT est polynomial, donc "simple" à résoudre. Nous allons voir qu'un algorithme probabiliste extrêmement naïf en vient (presque) à bout.<br><br>
<b>Algorithme WalkSat</b> <br>
<b>entrées :</b> Formule $F$ à $M$ clauses sur $n$ variables.<br>
<b>paramètre :</b> $T$ entier<br>
<b>sortie :</b> 
* Si c'est possible, renvoyer une affectation qui rende $F$ vraie
* Sinon, renvoyer `je n'ai pas trouvé`.

1. Initialisation : 
    - On tire une affectation `Affectation`$=(x_1,\dots,x_n)$ uniforme au hasard dans $\{\text{Vrai},\text{Faux}\}^n$
    - On pose Compteur $= 0$.
2. Tant que Compteur < $2Tn^2$  : 
  * 2a) Chercher la première clause non satisfaite
  * 2b) Dans cette clause, choisir une variable $x_i$ uniformément au hasard et la changer de valeur : $x_i \leftarrow \text{non}(x_i)$.
  * 2c) Si `Affectation` rend la formule vraie, renvoyer `Affectation`
  * 2d) Compteur = Compteur $+ 1$.
3. Renvoyer `je n'ai pas trouvé`

<i>(La raison pour laquelle on écrit le nombre de boucles sous la forme $2Tn^2$ apparaîtra à la partie suivante.)</i>


<div markdown=1 class=Rmk>

**Attention** L'algorithme WalkSat n'est pas tout à fait correct. Lorsque WalkSat renvoie `je n'ai pas trouvé`, cela ne signifie pas forcément que la formule n'est pas satisfiable mais peut-être simplement que l'algorithme n'a pas cherché assez longtemps.

<div markdown=1 class="DoIt"> 

Ecrire une fonction `UneEtapeWalkSat(n_var,Formule,Affectation)` qui effectue une fois les opérations 2a)-2b)-2c) dans la description de l'Algorithme WalkSat.

In [104]:
def UneEtapeWalkSat(n_var,Formule,Affectation):
    for c in Formule:
        if not ClauseVraie(c,Affectation,n_var):
            i = np.random.randint(0,len(c))
            idx = c[i]
            if idx<n_var:
                Affectation[idx] = int(not Affectation[idx])
            if idx>=n_var:
                Affectation[idx-n_var] = int(not Affectation[idx-n_var])
            break
    if FormuleVraie(Formule, Affectation, n_var):
        return Affectation
    return "Affectation ne rend pas la formule vraie"

In [107]:
UneEtapeWalkSat(2, Formule1, [1,0])

[1, 1]

<a id="ChoixT"></a>
## WalkSat : choix du paramètre $T$

Rappelons les propriétés suivantes de l'algorithme WalkSat appliqué à une formule $F$ à $n$ variables :
   - Si $F$ est fausse, l'algorithme renvoie `impossible`
   - Si $F$ est satisfiable
      * Avec une proba que l'on va noter $p(n,T,F,\mathbf{x})$ où $\mathbf{x}=(x_1,\dots,x_n)$ est l'affectation initiale, l'algorithme se trompe et renvoie `impossible`
      * Avec une proba $1-p(n,T,F,\mathbf{x})$ il renvoie une affectation correcte.
   
On note 
$$
p_\star(n,T) = \max_{F\text{ satisfiable},\mathbf{x}} p(n,T,F,\mathbf{x}) 
$$
où $\mathbf{x}=(x_1,\dots,x_n)$ est l'affectation initiale.

Tout l'enjeu est d'avoir une bonne majoration de $p_\star(n,T)$ en fonction de $T$.

Fixons une formule $F$ satisfiable, et fixons également une affectation $(y_1,\dots,y_n)$ correcte pour $F$. Pour $t\geq 0$ on note $(x_1^t,\dots,x_n^t)$ l'affectation de WalkSat à l'instant $t$.<br>
<!--<i>(Rappelons que $(x_1^0,\dots,x_n^0)$ sont uniformes indépendants.)</i><br>-->
Soit 
$$
D_t = \mathrm{card} \{ i \text{ tels que } x_i^t \neq y_i\}.
$$
Le processus $(D_t)_{t}$ est un processus aléatoire à valeurs dans $\{0,1,\dots,n\}$, lorsqu'il touche $0$ alors on a trouvé une affectation pour $F$. On introduit le temps aléatoire :

$$
\tau_E = \min\{t, (x_1^t,\dots,x_n^t)\text{ est une affectation correcte.}\}
$$

<div markdown=1 class="DoIt">

**(Théorie)** 

Soit $(S_t)_{t}$ le processus défini de la façon suivante :

   - $S_0=n$ 
   - Si $S_t=n$, alors $S_{t+1}=n-1$ 
   - Si $S_t=0$ alors $S_{t+1}=0$
   - Sinon $S_t$ vaut $S_t -1$ ou $S_t + 1$ avec probabilité $1/2$, indépendamment du passé.
   
Ainsi $(S_t)$ est la marche aléatoire symétrique sur l'intervalle $[0,n]$, réfléchie en $n$ et absorbée en $0$. On note 
<br>
<br>

$$
\tau_S = \min\{t, S_t=0\}.
$$

<br>
Démontrer que dans un certain sens $\tau_E$ a tendance à arriver plus tôt que $\tau_S$. Formellement, pour tout $t$,
<br><br>

$$
\mathbb{P}(\tau_E\geq t) \leq \mathbb{P}(\tau_S\geq t).
$$

<i>(Il n'est pas facile de rédiger soigneusement cette question, essayez plutôt de vous convaincre que c'est vrai!)</i>

<img src="TD08\figures\MarchesAleatoires.jpg" style="width:600px;"/>


<div markdown=1 class="Answers">

Intuitivement, de l'instant $t$ à l'instant $t+1$, on a que la probabilité que $D_{t+1} = D_t -1$ est supérieure à la probabilité que $S_{t+1} = S_t -1$. Cela vient du fait que:

$$
\mathbb{P}(S_{t+1} = S_t-1) = \frac{1}{2}
$$

Et, pour $D_t$, on a deux possibilités pour chaque clause qui est fausse: soit les deux éléments de la clause ne coincident pas avec l'affectation, soit un seul élément ne coincide pas. Alors, on a:

$$
\mathbb{P}(D_{t+1} = D_t-1) \geq \frac{1}{2}
$$

Donc on a la tendence d'avoir $D_t = 0$ avant d'avoir $S_t = 0$ at alors:

$$
\mathbb{P}(\tau_E\geq t) \leq \mathbb{P}(\tau_S\geq t).
$$

<div markdown=1 class="DoIt"> 

**(Théorie)**

Dans la section suivante vous allez vérifier numériquement que pour tout $n\geq 10$ alors
$$
\mathbb{P}(\tau_S\leq 2n^2) \geq  0.89.
$$
Grâce aux questions précédentes cela implique que
$$
\mathbb{P}(\tau_E\leq 2n^2) \geq  0.89.\qquad (\star)
$$

En utilisant $(\star)$, déterminer $T$ pour que pour toute formule satisfiable $F$ alors
$$
\mathbb{P}(\mathrm{WalkSat}\text{ trouve une affectation pour }F)\geq 1-\varepsilon.
$$
**Application numérique.**
Trouver $T$ pour que
$$
\mathbb{P}(\mathrm{WalkSat}\text{ trouve une affectation pour }F)\geq 99,99\%
$$

En sachant que $\tau_{E} = \min \{ t, (x_1^T, \dots, x_n^T)\}$, on a que:

Alors:
$$
\mathbb{P}(\text{WalkSat trouve une affectation pour } F) = \mathbb{P} (\tau_E < 2n^2 T) \\
\mathbb{P}(\text{WalkSat trouve une affectation pour } F) = 1 - \mathbb{P}(\tau_E > 2n^2 T) \\
\mathbb{P}(\text{WalkSat trouve une affectation pour } F) = 1 - \left(\mathbb{P}(\tau_E > 2n^2 )\right)^T
$$

Donc, pour avoir $\mathbb{P}(\mathrm{WalkSat}\text{ trouve une affectation pour }F)\geq 1-\varepsilon$ il faut que:

$$
T = \frac{\log{\varepsilon}} {\log \left(\mathbb{P}(\tau_E > 2n^2)\right)}
$$

En utilisant $(\star)$, on sait que $\mathbb{P}(\tau_E > 2n^2) \leq 0.11$. Alors, $T \geq \frac{\log{\varepsilon}} {\log \left(0.11\right)}$.

In [97]:
# Application numérique
epsilon = 1e-4

T = np.log(epsilon)/np.log(0.11)
T


4.172720088892946

<div markdown=1 class="DoIt"> 

**(Théorie)** Mais au fait, pourquoi la stratégie WalkSat ne marche pas pour $3$-SAT?

<div markdown=1 class="Answers"> 

<a id="Annexe"></a>
# Calcul de $\mathbb{P}(\tau_S\leq \lambda  𝑛^2)$ (matrices de transition)

Rappelons que $(S_t)$ est la marche aléatoire symétrique partant de $n$, réfléchie en $n$ et absorbée en $0$. On note $\tau_S = \min\{t, S_t=0\}$.

Pour déterminer $T$ nous avons eu besoin d'estimer numériquement la probabilité $\mathbb{P}(\tau_S\leq 2 𝑛^2)$. Nous allons pour cela utiliser une matrice de transition. Pour $t\geq 0$ et $0\leq i,j\leq n$ on note
<br><br>
$$
p_{i,j}^{(t)}=\mathbb{P}\left(\text{ en partant de $i$, }S_t=j\right).
$$


<div markdown=1 class="DoIt"> 

1. Soit $0\leq i\leq n$, $1\leq j\leq n-1$ et $t\geq 0$. Justifier que
$$
p^{(t)}_{i,j}=\tfrac{1}{2}p^{(t-1)}_{i,j-1}+\tfrac{1}{2}p^{(t-1)}_{i,j+1}.
$$
2. Réfléchir rapidement au cas $j=0$ ou $j=n$ dans l'équation suivante et en déduire qu'il existe une matrice $Q_n$ de taille $(n+1)\times (n+1)$telle que pour tous $t,i,j$,
$$
p^{(t)}_{i,j}= (Q_n^t)_{i,j}.
$$
Cette matrice $Q_n$ est appellée matrice de transition du processus $(S_t)$.


3. Ecrire $\mathbb{P}(\tau_S\leq t)$ en fonction de $Q_n$. En déduire un code python qui calcule $\mathbb{P}(\tau_S\leq 2 n^2)$ de façon exacte et vérifier sur un graphique que cette probabilité semble converger lorsque $n\to +\infty$. 

<div markdown=1 class="Answers"> 

1. Si on regarde la position $S_t = j$ dans la marche aléatoire, avec $1 < j < n$, on sait que, pour la position $S_{t-1}$ on a deux possibilités de même probabilité:

$$
\mathbb{P}(S_{t-1} = j+1) = \mathbb{P}(S_{t-1} = j-1) = \frac{1}{2}
$$

Alors on a: $p^{(t)}_{i,j}=\tfrac{1}{2}p^{(t-1)}_{i,j-1}+\tfrac{1}{2}p^{(t-1)}_{i,j+1}$

2. Maintenant on regarde les cas extrêmes.

Pour le cas $j = 0$, on a: $p^{(t)}_{i,0}=\frac{1}{2}p^{(t-1)}_{i,1}$

Pour le cas $j = n$, on a: $p^{(t)}_{i,n}=\frac{1}{2}p^{(t-1)}_{i,n-1}$

La matrice de transition est donc donnée par:

$$
    (Q_n^t)_{i,j} = \begin{cases} 
        \frac{1}{2} (Q_n^{(t-1)})_{i,j-1} + \frac{1}{2} (Q_n^{(t-1)})_{i,j+1} \text{, pour $0 < j < n$} \\
        \frac{1}{2} (Q_n^{(t-1)})_{i,j-1} \text{, pour $j = n$} \\
        \frac{1}{2} (Q_n^{(t-1)})_{i,j+1} \text{, pour $j = 0$} 
    \end{cases}
\text{,avec }(Q_n^0)_{i,j} = \mathbf{I}_{n+1}

$$

In [98]:
# Queston 3

T = int(2*n**2)
n = 5
Q = [np.zeros([n+1,n+1]) for i in range(T)]

Q[0] = np.eye(n+1)
for t in range(1,T):
    for i in range(n+1):
        for j in range(n+1):
            if j >1 and j < n-1:
                Q[t][i][j] = Q[t-1][i][j+1]*1/2 + Q[t-1][i][j-1]*1/2 
            elif j ==0 :
                Q[t][i][j] = Q[t-1][i][j+1]*1/2
            elif j == 1:
                Q[t][i][j] = Q[t-1][i][0] + Q[t-1][i][2]*1/2
            elif j == n:
                Q[t][i][j] = Q[t-1][i][j-1]*1/2
            elif j == n-1:
                Q[t][i][j] = Q[t-1][i][j-1]*1/2 + Q[t-1][i][j+1]

Q

[array([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]]),
 array([[0. , 1. , 0. , 0. , 0. , 0. ],
        [0.5, 0. , 0.5, 0. , 0. , 0. ],
        [0. , 0.5, 0. , 0.5, 0. , 0. ],
        [0. , 0. , 0.5, 0. , 0.5, 0. ],
        [0. , 0. , 0. , 0.5, 0. , 0.5],
        [0. , 0. , 0. , 0. , 1. , 0. ]]),
 array([[0.5 , 0.  , 0.5 , 0.  , 0.  , 0.  ],
        [0.  , 0.75, 0.  , 0.25, 0.  , 0.  ],
        [0.25, 0.  , 0.5 , 0.  , 0.25, 0.  ],
        [0.  , 0.25, 0.  , 0.5 , 0.  , 0.25],
        [0.  , 0.  , 0.25, 0.  , 0.75, 0.  ],
        [0.  , 0.  , 0.  , 0.5 , 0.  , 0.5 ]]),
 array([[0.   , 0.75 , 0.   , 0.25 , 0.   , 0.   ],
        [0.375, 0.   , 0.5  , 0.   , 0.125, 0.   ],
        [0.   , 0.5  , 0.   , 0.375, 0.   , 0.125],
        [0.125, 0.   , 0.375, 0.   , 0.5  , 0.   ],
        [0.   , 0.125, 0.   , 0.5  , 0.   , 0.375],
        [0.   

<a id="Transition"></a>
# Application : Illustration de la transition de phase

Le problème $2$-SAT aléatoire (lorsque les formules sont tirées aléatoirement et uniformément) présente un phénomène de <i>transition de phase</i>. Pour être plus formel nous introduisons quelques notations.<br><br>
Il y a $\binom{2n}{2}^M$ formules différentes avec $M$ clauses et $n$ variables (on considère que l'ordre des clauses compte, mais pas l'ordre des variables dans une clause). Notons
$$
p(n,M)
$$
la probabilité qu'une formule aléatoire uniforme parmi les $\binom{2n}{2}^M$ formules différentes soit vraie. 
On a bien sûr $p(n,M)$ qui est décroissante en $M$ (plus il y a de clauses plus c'est difficile d'être vraie).

Pour $c>0$ on a, lorsque $n\to +\infty$ :
$$
p(n,c\times n)\to
\begin{cases}
1 &\text{ si }c<1\\
0 &\text{ si }c>1
\end{cases}
$$

Référence : GENT, Ian P. et WALSH, Toby. The SAT phase transition. In : <i>ECAI</i>, 1994. p. 105-109.

<div markdown=1 class="DoIt"> 

1. Ecrire une fonction `TirerClause(k,nb_variables)` qui prend en entrées deux entiers $k,n$ et renvoie une clause de $k$-SAT à $n$ variables, uniformément au hasard.
2. Ecrire une fonction `TirerFormule(k,nb_variables,M)` qui renvoie une formule de $M$ clauses de $k$-SAT à $n$ variables, uniformément au hasard.

<br>
<i>(On considère qu'il y a $\binom{2n}{2}^M$ formules différentes avec $M$ clauses et $n$ variables : l'ordre des clauses compte, mais pas l'ordre des variables dans une clause.)</i>

<div markdown=1 class="DoIt"> 

1. Ecrire un solveur WalkSat qui avec le paramètre $T$ choisi à l'exercice précédent.
2. En utilisant des simulations et le solveur WalkSat, illustrer la transition de phase en $c=1$.